In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.interpolate import interpolate_to_grid, remove_nan_observations
from metpy.plots import add_metpy_logo
from metpy.units import units

### Set up a map projection to use
to_proj = ccrs.AlbersEqualArea(central_longitude=-97., central_latitude=38.)

In [ ]:
### Load some test data included in metpy.cbook
with get_test_data('station_data.txt') as f:
    data = pd.read_csv(f, header=0, usecols=(2, 3, 4, 5, 18, 19),
                       names=['latitude', 'longitude', 'slp', 'temperature', 'wind_dir',
                              'wind_speed'],
                       na_values=-99999)

In [ ]:
### View the data
data

In [ ]:
### extract the lat and lon coordinates and then transform them to geodetic coords
lon = data['longitude'].values
lat = data['latitude'].values
xp, yp, _ = to_proj.transform_points(ccrs.Geodetic(), lon, lat).T

In [ ]:
### Check the transformation
print('lon: ',lon[0])
print('lat: ',lat[0])
print('xp:  ',xp[0])
print('yp:  ',yp[0])

In [ ]:
### Make a plot to view the station locations
fig = plt.figure(figsize=(20, 10))
view = fig.add_subplot(1, 1, 1, projection=to_proj)

view.set_extent([-120, -70, 20, 50])
view.add_feature(cfeature.STATES.with_scale('50m'))
view.add_feature(cfeature.OCEAN)
view.add_feature(cfeature.COASTLINE.with_scale('50m'))
view.add_feature(cfeature.BORDERS, linestyle=':')

view.scatter(xp,yp,c='blue')

plt.show()

In [ ]:
### Clean up the pressure data by removing missing observations (NaNs)
x_masked, y_masked, pressure = remove_nan_observations(xp, yp, data['slp'].values)

In [ ]:
### Plot the stations with valid pressure data
fig = plt.figure(figsize=(20, 10))
view = fig.add_subplot(1, 1, 1, projection=to_proj)

view.set_extent([-120, -70, 20, 50])
view.add_feature(cfeature.STATES.with_scale('50m'))
view.add_feature(cfeature.OCEAN)
view.add_feature(cfeature.COASTLINE.with_scale('50m'))
view.add_feature(cfeature.BORDERS, linestyle=':')

scatter = view.scatter(x_masked,y_masked,c=pressure) #Color code the station by its pressure value
cbar = plt.colorbar(scatter)
cbar.set_label('Pressure (mb)')

plt.show()

In [ ]:
### Interpolate pressure observations to a grid
slpgridx, slpgridy, slp = interpolate_to_grid(x_masked, y_masked, pressure,
                                              interp_type='cressman', minimum_neighbors=1,
                                              search_radius=400000, hres=100000) # the hres changes the grid resolution

In [ ]:
### Make the same plot, but now with the interpolated pressure grid contoured over the stations
fig = plt.figure(figsize=(20, 10))
view = fig.add_subplot(1, 1, 1, projection=to_proj)

view.set_extent([-120, -70, 20, 50])
view.add_feature(cfeature.STATES.with_scale('50m'))
view.add_feature(cfeature.OCEAN)
view.add_feature(cfeature.COASTLINE.with_scale('50m'))
view.add_feature(cfeature.BORDERS, linestyle=':')

scatter = view.scatter(x_masked,y_masked,c=pressure)
cbar = plt.colorbar(scatter)
cbar.set_label('Pressure (mb)')

cs = view.contour(slpgridx, slpgridy, slp, colors='k', levels=list(range(990, 1034, 4)))
view.clabel(cs, inline=1, fontsize=12, fmt='%i')

plt.show()

In [ ]:
### Plot the resulting pressure grid with the station locations overlaid
fig = plt.figure(figsize=(20, 10))
view = fig.add_subplot(1, 1, 1, projection=to_proj)

view.set_extent([-120, -70, 20, 50])
view.add_feature(cfeature.STATES.with_scale('50m'))
view.add_feature(cfeature.OCEAN)
view.add_feature(cfeature.COASTLINE.with_scale('50m'))
view.add_feature(cfeature.BORDERS, linestyle=':')

cf = view.contourf(slpgridx, slpgridy, slp, levels=list(range(990, 1034, 4)))
cbar = plt.colorbar(cf)
cbar.set_label('Pressure (mb)')
view.scatter(x_masked,y_masked,c='black')

plt.show()

In [ ]:
### See the resulting grid shape
slp.shape

In [ ]:
### On your own: Repeat with temperature observations.